## GLOVE
word2vec은 윈도우 크기로 주변 단어를 고려하기 때문에 전체 맥락은 반영할 수 없다.<br>
그에 비해 glove는 전체 정보를 반영과 단어 의미 유추를 함께 할수 있는 임베딩 방법이다.

In [1]:
!pip install glove_python

     |████████████████████████████████| 266kB 5.3MB/s 
  Created wheel for glove-python: filename=glove_python-0.1.0-cp36-cp36m-linux_x86_64.whl size=700220 sha256=485d9a7ea4131b61eb64a42f37e6f276775566da5253157eb9ed60ee8e9849a7
  Stored in directory: /root/.cache/pip/wheels/88/4b/6d/10c0d2ad32c9d9d68beec9694a6f0b6e83ab1662a90a089a4b
Successfully built glove-python


In [4]:
import csv
import re

def open_csv():
    # csv파일을 연다!
    f = open('/content/drive/MyDrive/NLP/data/IMDB Dataset.csv', 'r', encoding='utf-8')
    csvreader = csv.reader(f)
    
    doc_list = []

    next(csvreader)
    for f in csvreader:
        line = re.compile("[^\w]").sub(' ', f[0].lower())
        doc_list.append(line.split())

    return doc_list

doc_list = open_csv()

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
from glove import Corpus, Glove

corpus = Corpus()
# 같이 등장한다는 것의 기준을 5로 지정
corpus.fit(doc_list, window=5)

# 경사하강법을 사용하기 때문에 학습률을 설정, 아웃풋 벡터의 차원은 100
glove = Glove(no_components=100, learning_rate=0.05)
# 쓰레드 개수는 4개, 훈련 횟수는 20번, verbose 설명
glove.fit(corpus.matrix, epochs=20, no_threads=4, verbose=True)
# 유사도 검색을 위해서 행렬의 index 정보를 입력
glove.add_dictionary(corpus.dictionary)

Performing 20 training epochs with 4 threads
Epoch 0
Epoch 1
Epoch 2
Epoch 3
Epoch 4
Epoch 5
Epoch 6
Epoch 7
Epoch 8
Epoch 9
Epoch 10
Epoch 11
Epoch 12
Epoch 13
Epoch 14
Epoch 15
Epoch 16
Epoch 17
Epoch 18
Epoch 19


In [6]:
glove_result = glove.most_similar('dog')
print(glove_result)

[('baby', 0.8120328673602765), ('horse', 0.7840698411819933), ('cat', 0.7811717330678664), ('kid', 0.7617122533703016)]


In [7]:
from tensorflow.keras.preprocessing.text import Tokenizer

token = Tokenizer()
text = "Tensorflow tokenizer is good"

# 리스트 형태로 넣어주어야 제대로 토큰화가 된다 (아니면 한글자씩 짜름)
# fit 한 것을 토대로 추후에 sequence 를 생성한다
token.fit_on_texts([text])

# 단어 집합
print(token.word_index)
# 단어 빈도
print(token.word_counts)

# 정수 인코딩
text_2 = "Tensorflow tokenizer is not good good"
int_encoding = token.texts_to_sequences([text_2])
print(int_encoding)

# 행렬화
text_list = ['안녕하세요 감사해요 잘있어요 다시 만나요', '오늘도 기분 좋은 하루', '안녕하세요 오늘도 감사해요']
token.fit_on_texts(text_list)

# count 는 dtm, binary 는 존재 여부, freq 는 빈도율, tfidf 는 tfidf
matrix = token.texts_to_matrix(text_list, mode='tfidf')
print(matrix)

{'tensorflow': 1, 'tokenizer': 2, 'is': 3, 'good': 4}
OrderedDict([('tensorflow', 1), ('tokenizer', 1), ('is', 1), ('good', 1)])
[[1, 2, 3, 4, 4]]
[[0.         0.84729786 0.84729786 0.         0.         0.
  0.         0.         1.09861229 1.09861229 1.09861229 0.
  0.         0.        ]
 [0.         0.         0.         0.84729786 0.         0.
  0.         0.         0.         0.         0.         1.09861229
  1.09861229 1.09861229]
 [0.         0.84729786 0.84729786 0.84729786 0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.        ]]
